In [1]:
import sqlite3
import pandas as pd
from sqlite3 import connect
conn = sqlite3.connect('tanks-4-09-data.db')

In [2]:
solar_data = pd.read_csv('table_7_1_6_solarabs.csv')
met_data = pd.read_sql('SELECT * FROM met_data', conn)
chem_data = pd.read_csv('input_data/chemical_db.csv')
revised_met_data = pd.read_csv('revised-met-data-2020.csv')

#revised_met_data.to_sql('revised_met_data', conn, if_exists='replace', index=False)
#solar_data.to_sql('solar_abs_data', conn, if_exists='replace', index=False)
#met_data.to_sql('met_data', conn, if_exists='replace', index=False)
#chem_data.to_sql('chemical_data', conn, if_exists='replace', index=False)

In [22]:
queryStringProduct1 = "SELECT * FROM chemical_data LIMIT 10" 
queryStringProduct2 = "WHERE CATEGORY == " + "'"  
queryStringProduct = queryStringProduct1  
sqlQueryProduct = pd.read_sql(queryStringProduct, conn)
sqlQueryProduct[['NAME', 'CAS', 'VP_MOLWT', 'MOLWT', 'REID', 'VP_COEF_A', 'VP_COEF_B', 'VP_COEF_C']]

,NAME,CAS,VP_MOLWT,MOLWT,REID,VP_COEF_A,VP_COEF_B,VP_COEF_C
0,Acetaldehyde,00075-07-0,44.00,44.00,0.0,8.0050,1600.017,291.809
1,Acetic acid,00064-19-7,60.05,60.05,0.0,7.3870,1533.313,222.309
2,Acetic anhydride,00108-24-7,102.09,102.09,0.0,7.1490,1444.718,199.817
3,Acetone,00067-64-1,58.08,58.08,0.0,7.1170,1210.595,229.664
4,Acetonitrile,00075-05-8,41.05,41.05,0.0,7.1190,1314.400,230.000
5,Acrylamide,00079-06-1,71.09,71.09,0.0,11.2932,3939.877,273.160
6,Acrylic acid,00079-10-7,72.10,72.10,0.0,8.5387,2305.843,266.547
7,Acrylonitrile,00107-13-1,53.06,53.06,0.0,7.0380,1232.530,222.470
8,Allyl alcohol,00107-18-6,58.08,58.08,0.0,0.0000,0.000,0.000
9,Allyl chloride,00107-05-1,76.53,76.53,0.0,0.0000,0.000,0.000


In [33]:
imput_chemical_data = [{'name': 'Acrylonitrile', 'composition': 0.30}, 
                       {'name': 'Acetone', 'composition': 0.60}, 
                       {'name': 'Acetic anhydride', 'composition': 0.10}]

name_list = [each['name'] for each in imput_chemical_data]
comp_list = [each['composition'] for each in imput_chemical_data]

if sum(comp_list) >= 0.999999:
    
    strBuild = "("
    for idx, each in enumerate(name_list[:-1]):
        strBuild += "'" + each + "'"  + ', '
    whereString = strBuild + "'" + name_list[-1]+ "'" + ")"

    queryStringProductOL1 = "SELECT * FROM chemical_data "
    queryStringProductOL2 = "WHERE name IN " + whereString
    queryStringProductOL3 = queryStringProduct1 + queryStringProduct2
    sqlQueryProductOL = pd.read_sql(queryStringProductOL3, conn)
    queryProductOL = sqlQueryProductOL[['NAME', 'CAS', 'VP_MOLWT', 'MOLWT', 
                                         'REID', 'VP_COEF_A', 'VP_COEF_B', 
                                         'VP_COEF_C']]
    productDataOL = queryProductOL.assign(composition = comp_list)
    
else:
    raise ValueError('Not 100%') 

In [35]:
tlx_averageDailyMaxLiqSurfaceTemp_R = 500
tln_averageDailyMinLiqSurfaceTemp_R = 500
tla_averageDailyLiquidTemp = 500



check_A = productDataOL['VP_COEF_A'].values.tolist()
check_B = productDataOL['VP_COEF_B'].values.tolist()
check_C = productDataOL['VP_COEF_C'].values.tolist()

if 0 not in check_A:
    if 0 not in check_B:
        if 0 not in check_C:

            # tlx_averageDailyMaxLiqSurfaceTemp_R
            productDataOL['log_pva'] = productDataOL['VP_COEF_A'] - ((productDataOL['VP_COEF_B']) /\
                                                                           (tlx_averageDailyMaxLiqSurfaceTemp_R + productDataOL['VP_COEF_C']))
            productDataOL['pva'] = 10**(productDataOL['log_pva'])

            productDataOL['frac_pva'] = productDataOL['pva'] * productDataOL['composition']
            productDataOL['moles'] = 100*productDataOL['composition'] / productDataOL['MOLWT']
            
            totalMoles = sum(productDataOL['moles'].values.tolist())
            
            productDataOL['mol_frac'] = productDataOL['moles'] / totalMoles
            productDataOL['partial_pressure'] = productDataOL['mol_frac'] * productDataOL['pva']
            
            pva = sum(productDataOL['partial_pressure'].values.tolist())
            
            productDataOL['vap_mole_frac'] = productDataOL['partial_pressure'] / pva
            productDataOL['mol_wt_frac'] = productDataOL['vap_mole_frac'] * productDataOL['VP_MOLWT']
            
            mv = sum(productDataOL['mol_wt_frac'].values.tolist())

        else:
            raise ValueError('Value C Not Present in Data')
    else:
        raise ValueError('Value B Not Present in Data')
else: 
    raise ValueError('Value A Not Present in Data')

In [36]:
mv

61.751081892433824

In [ ]:
# Put in city, and get all of the TAX values.
locationString1 = 'Long Beach, CA'
locationString2 = "SELECT * FROM revised_met_data WHERE location == " 
locationString3 = "'" + str(locationString1) + "'"
queryStringTax = locationString2 + locationString3 
sqlQueryTax = pd.read_sql(queryStringTax, conn)
sqlQueryTax[sqlQueryTax['symbol'] == 'TAX'].values.tolist()[0][3:16]

In [ ]:
# Put in city, and get all of the TAX values.
locationString1 = 'Long Beach, CA'
locationString2 = "SELECT * FROM revised_met_data WHERE location == "
locationString3 = "'" + str(locationString1) + "'"
queryStringAtmPress = locationString2 + locationString3 
sqlQueryAtmPress = pd.read_sql(queryStringAtmPress, conn)
sqlQueryAtmPress[sqlQueryAtmPress['symbol'] == 'TAN'].values.tolist()[0][3:16]

In [ ]:
locationString = 'Long Beach, California'
queryStringAtmPress = "SELECT * FROM met_data WHERE CTYST == " + "'" + str(locationString) + "'"
sqlQueryAtmPress = pd.read_sql(queryStringAtmPress, conn)

In [ ]:
list(set(pd.read_sql('SELECT * FROM chemical_data', conn)['CATEGORY'].tolist()))

In [ ]:

#productClassString = 'Petroleum Distillates'
productNameString = 'Crude oil (RVP 5)'
queryStringProduct = "SELECT * FROM chemical_data WHERE CATEGORY == " + "'" + str(productClassString) + "'"
sqlQueryProduct = pd.read_sql(queryStringProduct, conn)
rvp = sqlQueryProduct['REID'].values.tolist()[0]
#rvp = sqlQueryProduct

In [ ]:
rvp

In [ ]:
productClassString = 'Crude Oils'
#productNameString = 'Crude oil (RVP 5)'
queryStringProduct1 = "SELECT * FROM chemical_data " 
queryStringProduct2 = "WHERE CATEGORY == " + "'"  
queryStringProduct3 = str(productClassString) + "'"
queryStringProduct = queryStringProduct1 + queryStringProduct2 + queryStringProduct3
sqlQueryProduct = pd.read_sql(queryStringProduct, conn)
#sqlQueryProductData = sqlQueryProduct[sqlQueryProduct['NAME'] == productNameString]
#sqlQueryProductRVP = sqlQueryProductData['REID'].values.tolist()[0]
sqlQueryProduct

In [ ]:
sqlQueryProduct[(sqlQueryProduct['CATEGORY'] == 'Petroleum Distillates') & (sqlQueryProduct['REID'] > 0)]['NAME'].values.tolist()

In [ ]:
queryStringProduct1 = "SELECT * FROM chemical_data " 
queryStringProduct2 = "WHERE CATEGORY == " + "'"  
queryStringProduct = queryStringProduct1 + queryStringProduct2 
sqlQueryProduct = pd.read_sql(queryStringProduct, conn)

In [70]:
class Grant:
    def __init__(self):
        pass
    
    def func1(self):
        self.x = 900
        return self.x
    
    def func2(self):
        return self.x

In [71]:
ggg = Grant()

In [72]:
ggg.func2()

AttributeError: 'Grant' object has no attribute 'x'